# Yuan 2004 "New" Algorithm

See: [A new algorithm to get the initial centroids](https://ieeexplore.ieee.org/abstract/document/1382371)

In [1]:
import imports
import numpy as np
from scipy.spatial import distance as spdistance
import sklearn.datasets as skdatasets
import sklearn.metrics as skmetrics 
import kmeans

In [2]:
''''data = np.array([
    [4,4,3,4],
    [1,1,2,1],
    [7,7,7,7], 
    [1,1,1,1],
    [7,8,9,7],
    [5,5,5,5],
    [100,100,100,100]
])'''

dataset = skdatasets.load_iris()
data = dataset.data
target = dataset.target

K = 3

### Find distances between all rows

In [3]:
#TODO: this is brute force and won't scale. Try to find better (vectorised?) solution

def distance_table(data):
    numrows = len(data)
    numcols = len(data[0])

    distances = np.nan * np.empty((numrows, numrows))

    for i in range(0, numrows):
        for j in range(0, numrows):
            if i != j: 
                distances[i][j] = spdistance.euclidean(data[i], data[j])

    return distances
        
distances = distance_table(data)
print(distances)

[[       nan 0.53851648 0.50990195 ... 4.45982062 4.65080638 4.14004831]
 [0.53851648        nan 0.3        ... 4.49888875 4.71805044 4.15331193]
 [0.50990195 0.3               nan ... 4.66154481 4.84871117 4.29883705]
 ...
 [4.45982062 4.49888875 4.66154481 ...        nan 0.6164414  0.64031242]
 [4.65080638 4.71805044 4.84871117 ... 0.6164414         nan 0.76811457]
 [4.14004831 4.15331193 4.29883705 ... 0.64031242 0.76811457        nan]]


### Find closest two rows in U

In [4]:
def find_closest(data):
    
    distances = distance_table(data)
    ind = np.unravel_index(np.nanargmin(distances, axis=None), distances.shape)

    return list(ind)

print(find_closest(data))

[101, 142]


### TODO: find next closest

In [5]:

def find_next_closest(Am, U):

    pass

### Main loops

In [6]:
# Holder for the point sets
A = []

U = data.copy()

while len(A) < K:

    Am = []
    pair = find_closest(U)
    
    Am.append(U[pair[0]])
    Am.append(U[pair[1]])

    U = np.delete(U, list(pair), 0)
  
    ## TODO:
    ##while len(Am) < THAT_SPECIAL_NUMBER
    ##    next = find_next_closest(U, Am)
    ##    #append
    ##    #delete
        
    A.append(Am)
    
    
print(A)
#print(U)

[[array([5.8, 2.7, 5.1, 1.9]), array([5.8, 2.7, 5.1, 1.9])], [array([5. , 3.4, 1.5, 0.2]), array([5.1, 3.4, 1.5, 0.2])], [array([5.1, 3.5, 1.4, 0.2]), array([5.1, 3.5, 1.4, 0.3])]]


### Sum the vectors in U

In [7]:
centroids = []

for pointset in A:
    centroids.append(np.mean(pointset, 0))
        
centroids = np.array(centroids)
print(centroids)

[[5.8  2.7  5.1  1.9 ]
 [5.05 3.4  1.5  0.2 ]
 [5.1  3.5  1.4  0.25]]


### Run k-means

In [8]:
Z, U, clusters, iterations = kmeans.cluster(data, K, centroids.copy())

print(U)
print(target)

[2 1 1 1 2 2 1 2 1 1 2 2 1 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 1 1 2 2 2 1 2 2
 2 1 2 2 1 1 2 2 1 2 1 2 2 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### Run some metrics

In [9]:
acc = skmetrics.accuracy_score(target, U)
ari = skmetrics.adjusted_rand_score(target, U)

print("Accuracy Score:", acc)
print("Adjusted Rand Index:", ari)

Accuracy Score: 0.02666666666666667
Adjusted Rand Index: 0.4289511167236898
